In [26]:
import pandas as pd

def detectar_alertas_automaticamente_csv(path_csv, comercio_objetivo, mes_objetivo):
    """
    Carga un archivo CSV, estandariza nombres de comercio, asigna giros,
    calcula moda por cliente y comercio, y detecta alertas para un comercio y mes específico.
    """

    # Cargar archivo CSV
    df = pd.read_csv(path_csv)

    # Reemplazos para estandarizar nombres de comercios
    reemplazos = {
        'SMARTFIT': 'SMART FIT', 'SMART FIT': 'SMART FIT',
        'TOTAL PLAY': 'TOTALPLAY', 'TOTALPLAY': 'TOTALPLAY',
        'GOOGLE YOUTUBE': 'GOOGLE',
        'DIDI FOOD': 'DIDIFOOD', 'DIDIFOOD': 'DIDIFOOD',
        'WAL-MART': 'WALMART',
        'MERCADOPAGO': 'MERCADO PAGO',
        'MI ATT': 'AT&T', 'ATT': 'AT&T',
        'TOTALPASS': 'TOTAL PASS'
    }
    df['comercio'] = df['comercio'].replace(reemplazos)

    # Clasificación por giros
    giros = {
        'OXXO': 'Conveniencia', '7 ELEVEN': 'Conveniencia', '7ELEVEN': 'Conveniencia',
        'OXXO GAS': 'Gasolineras', 'COSTCO GAS': 'Gasolineras',
        'WALMART': 'Supermercado', 'SUPERCENTER': 'Supermercado', 'SUPERAMA': 'Supermercado',
        'SORIANA': 'Supermercado', 'HEB': 'Supermercado', 'CHEDRAUI': 'Supermercado', 'ALSUPER': 'Supermercado',
        'SAMS CLUB': 'Club de mayoreo', 'COSTCO': 'Club de mayoreo',
        'TELCEL': 'Telefonía móvil', 'AT&T': 'Telefonía móvil', 'BAIT': 'Telefonía móvil', 'TELEFONICA': 'Telefonía móvil',
        'TELMEX': 'Internet / TV de paga', 'TOTALPLAY': 'Internet / TV de paga', 'IZZI': 'Internet / TV de paga',
        'MEGACABLE': 'Internet / TV de paga', 'CABLEYCOMUN': 'Internet / TV de paga',
        'NETFLIX': 'Streaming de video', 'DISNEY PLUS': 'Streaming de video', 'AMAZON PRIME': 'Streaming de video',
        'CRUNCHYROLL': 'Streaming de video', 'MAX': 'Streaming de video', 'VIX': 'Streaming de video', 'ROKU': 'Streaming de video',
        'SPOTIFY': 'Streaming de música', 'ITUNES': 'Streaming de música', 'AUDIBLE': 'Streaming de música',
        'GOOGLE YOUTUBEPREMIUM': 'Streaming de música', 'GOOGLE YOUTUBE': 'Streaming de música',
        'OPENAI': 'Suscripción digital', 'GOOGLE ONE': 'Suscripción digital', 'APPLE': 'Suscripción digital',
        'MICROSOFT': 'Suscripción digital', 'TOTAL PASS': 'Suscripción digital',
        'ADOBE': 'Plataforma de diseño', 'CANVA': 'Plataforma de diseño',
        'UBER': 'Movilidad', 'DIDI': 'Movilidad', 'DIDI RIDES': 'Movilidad',
        'UBER EATS': 'Delivery de comida', 'DIDI FOOD': 'Delivery de comida', 'DIDIFOOD': 'Delivery de comida',
        'RAPPI': 'Delivery de comida', 'SOFT RAPPI': 'Delivery de comida',
        'RAPPIPRO': 'Suscripción de comida',
        'FARMACIAS DEL AHORRO': 'Farmacia', 'FARMACIAS GUADALAJARA': 'Farmacia', 'FARMACIAS SIMILARES': 'Farmacia',
        'STARBUCKS': 'Restaurante / Cafetería', 'CARLS JR': 'Restaurante / Cafetería', 'MELIMAS': 'Restaurante / Cafetería',
        'LIVERPOOL': 'Retail', 'SEARS': 'Retail', 'COPPEL': 'Retail',
        'CALIENTE': 'Apuestas y loterías', 'TULOTERO': 'Apuestas y loterías', 'BET365': 'Apuestas y loterías',
        'CFE': 'Servicios públicos', 'SERV AGUA DREN': 'Servicios públicos',
        'KUESKI PAY': 'Fintech / BNPL', 'APLAZO': 'Fintech / BNPL', 'APLAZ': 'Fintech / BNPL', 'CASHI ECOMMERCE': 'Fintech / BNPL',
        'MERCADO PAGO': 'Pasarela de pago',
        'AMAZON': 'Marketplace / E-commerce', 'ALIEXPRESS': 'Marketplace / E-commerce', 'TEMU': 'Marketplace / E-commerce',
        'MI ATT': 'Gestión de pagos móviles', 'RENTAMOVISTAR': 'Gestión de pagos móviles', 'UBRPAGOSMEX': 'Gestión de pagos móviles',
        'METROBUS': 'Transporte', 'URBANI': 'Transporte', 'PARCO': 'Transporte',
        'FACEBOOK': 'Otros', 'NAYAX': 'Otros', 'BAE': 'Otros', 'SMART': 'Otros', 'ROTOPLAS': 'Otros', 'PLAYSTATION NETWORK': 'Otros'
    }
    df['giro_DA'] = df['comercio'].map(giros).fillna('Sin clasificar')

    # Procesar fechas y crear columna 'anio_mes'
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['anio_mes'] = df['fecha'].dt.to_period('M').astype(str)

    # Calcular moda por cliente y comercio
    df_moda = (
        df.groupby(['id', 'comercio'])['monto']
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
        .reset_index()
        .rename(columns={'monto': 'monto_moda'})
    )
    dict_moda = df_moda.set_index(['id', 'comercio'])['monto_moda'].to_dict()

    # Calcular diferencia y alerta
    df['diferencia_vs_moda'] = df.apply(
        lambda row: row['monto'] - dict_moda.get((row['id'], row['comercio']), 0), axis=1
    )
    df['alerta'] = df['diferencia_vs_moda'] > 0

    # Filtrar resultados
    df_resultado = df[
        (df['comercio'].str.lower() == comercio_objetivo.lower()) &
        (df['anio_mes'] == mes_objetivo) &
        (df['alerta'])
    ]

    return df_resultado[['id', 'comercio', 'anio_mes', 'monto', 'diferencia_vs_moda', 'alerta', 'giro_DA']], df

In [27]:
ruta_csv = '/content/base_transacciones_final(in).csv'

df_alertas, df = detectar_alertas_automaticamente_csv(
    path_csv=ruta_csv,
    comercio_objetivo='netflix',
    mes_objetivo='2022-12'
)

print(df_alertas)

                                              id comercio anio_mes  monto  \
7069    8d2d5fdb0101dbecdbdbd7e9bf92e59f04233cae  NETFLIX  2022-12  19.66   
7331    798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-12  38.04   
12409   62963834e157b2e52e0db4f39171a160ab243f30  NETFLIX  2022-12  13.91   
15963   c59a9663a700f80b76dfe4e9d40122cb404110c1  NETFLIX  2022-12  28.85   
17001   3e86f3f66a96699fe2a84164892b66138b87c333  NETFLIX  2022-12  13.91   
...                                          ...      ...      ...    ...   
325438  d359ab417e34fe192e8d846b6427d2f764f46683  NETFLIX  2022-12  45.96   
331317  298d41a93ab1b0da3975e94d3fe4fa179b147ac5  NETFLIX  2022-12  13.91   
333608  38aea4f15b9ddd201ad4126c09ac898ff4fa26fd  NETFLIX  2022-12  28.85   
334520  5a1c6d0a09d4fa2fffad2c2f13ad73960a0f9c5e  NETFLIX  2022-12  38.04   
345966  710f13196a0f9109e22bc73c2fe76f22fbef3dc1  NETFLIX  2022-12  19.66   

        diferencia_vs_moda  alerta             giro_DA  
7069              

In [28]:
df_ejemplo = df[
    (df['comercio'].str.lower() == 'netflix') &
    (df['id'] == '798aaf7e3d3e55f454267e32ab64b52f87d84567')
]
print(df_ejemplo[['id', 'comercio', 'anio_mes', 'monto', 'diferencia_vs_moda']])

                                            id comercio anio_mes  monto  \
7104  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-01  34.59   
7120  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-02  34.59   
7142  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-03  34.59   
7173  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-04  34.59   
7199  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-05  34.59   
7218  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-06  34.59   
7231  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-07  34.59   
7244  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-08  34.59   
7264  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-09  34.59   
7286  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-10  34.59   
7309  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-11  34.59   
7331  798aaf7e3d3e55f454267e32ab64b52f87d84567  NETFLIX  2022-12  38.04   
7356  798aaf7e3d3e55f4542